# Manoeuvring simulations
Many simulation model for ship manoeuvring have been developed in the field of ship hydrodynamics such as: the Abkowitz model {cite:p}`abkowitz_ship_1964` or the Norrbin model {cite:p}`norrbin_study_1960`.
This chapter will develop a general simulation model for ship manoeuvring, that can be further specified to become either the Abkowitz or Norbin model. Expressing the models on a general form is important in this research where many different models will be tested and compared.

In [ ]:
import matplotlib 
print(matplotlib.get_configdir())

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex, Markdown
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from myst_nb import glue
from vessel_manoeuvring_models.symbols import *

import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot, plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_nonlinear_EOM  as vmm 

import matplotlib.pyplot as plt
if os.name == 'nt':
    plt.style.use('book.mplstyle')  # Windows
    
## Examples
from example_1 import ship_parameters, df_parameters
from myst_nb import glue

p = df_parameters["symbol"]

In [ ]:
symbols.glue_equations(symbols)

In [ ]:
glue('eq_6DOF',Math(vlatex(eq_6DOF)))

In [ ]:
glue('eq_eta',Math(vlatex(eq_eta)))

In [ ]:
glue('eq_nu',Math(vlatex(eq_nu)))

The models can be expressed in a very general way as shown in {numref}`eq_6DOF` ( {cite:p}`fossen_handbook_2021`).

```{glue:math} eq_6DOF
:label: eq_6DOF
```

Where $\eta$ describes the position and $\nu$ is the velocities:

```{glue:math} eq_eta
:label: eq_eta
```

```{glue:math} eq_nu
:label: eq_nu
```

The model matrixes:
* $M$ : inertia
* $C$ : corriolis/centrepetal
* $D$ : damping
* $g(\eta)$ is a vector of generalized gravitational an buoyance forces.
* $g_0$ is static restoring forces due to ballast systems.

The inertia as well as the corriolis matrix can be split into actual mass/intertia and added mass/inertia:

```{glue:math} eq_M
:label: eq_M
```

```{glue:math} eq_C
:label: eq_C
```

So that the model equation can be written as:
```{glue:math} eq_6DOF_expanded
:label: eq_6DOF_expanded
```

The velocity of the current can be described as (assumed to be irrotational):
```{glue:math} eq_nu_r
:label: eq_nu_c
```

So that $\nu_r$ are the relative velocities (including current):
```{glue:math} eq_nu_r
:label: eq_nu_r
```

If the current is also assumed to be constant, acceleration of the relative velocities is the same:
```{glue:math} eq_nu_steady
:label: eq_nu_steady
```
So that the entire equation can be expressed in terms of relative velocities:

```{glue:math} eq_6DOF_expanded_r
:label: eq_6DOF_expanded_r
```




## Ship parameters
The Ocean Bird Wind Powered Car Carrier will be used as the reference ship of this book:
![](https://www.walleniusmarine.com/wp-content/uploads/2020/09/oceanbird-960x540.jpg)

In [ ]:
ps = prime_system.PrimeSystem(**ship_parameters)  # model

A model scale version of this ship has the following main dimensions:

In [ ]:
ship_parameters

## 3 DOF
A simulation model with only three degrees of fredom (DOF) : *surge*, *sway* and *yaw* is often sufficient to describe the ship's manoeuvring performance. The equation of motion can be expressed as <cite id="2k4ro">(Triantafyllou &#38; Hover, 2003)</cite>:

In [ ]:
display(vmm.X_eom)
display(vmm.Y_eom)
display(vmm.N_eom)


Where $X_{force}$, $Y_{force}$ and $N_{force}$ are the hydrodynamic forces from the ship. So these equations holds for most of the different models for manoeuvring, except linear models where the nonlinear $r\cdot u$ term have been linearized. The difference in the methods is rather in how these hydrodynamic forces are calculated.

The hydrodynamic forces can be split into added masses (that depend on the accelerations: $\dot{u}$, $\dot{v}$, $\dot{r}$) and dampings (that depend on the velocities: $u$, $v$, $r$):

In [ ]:
display(Math(vlatex(vmm.fx_eq)))
display(Math(vlatex(vmm.fy_eq)))
display(Math(vlatex(vmm.mz_eq)))

$X_{\dot{u}}$, $X_{\dot{v}}$, $X_{\dot{r}}$ are the added masses: when the ship is accelerating in water, not only the mass of the ship is accelerating, but also some mass in the water needs to be accelerated. This is referred to as added masses. Sometimes added  masses such as $X_{\dot{v}}$ is neglected, as in the case above.

The specialization of the various simulation models can now be further categorized to the functions to calculate the damping forces: $X_{D}$, $Y_{D}$, $N_{D}$.

In [ ]:
ship_parameters_prime = ps.prime(ship_parameters)

The general equation for all of the simulation models in this research can be written as: 

In [ ]:
display(vmm.X_eq)
display(vmm.Y_eq)
display(vmm.N_eq)

The classic simulation methods express the damping forces as a truncated taylor series. Here is a rather simple model, that has been created by the author to be used in the followin examples.

In [ ]:
display(vmm.X_qs_eq)
display(vmm.Y_qs_eq)
display(vmm.N_qs_eq)

## Similitude

### Prime system
The prime system uses ship length $L$, water density $\rho$ and total velocity $U^2=u^2+v^2$ to express physical quantities in nondimensional form. The quantities have a prime symbol (') attached to them when the prime system is used. The quantities are made nondimensional with the prime system by dividing with a denominator according to the table below:

In [ ]:
prime_system.df_prime.loc[['denominator']].transpose()

The manoeuvring models are often expressed in the prime system so that a nondimensional force $F'$ for instance can be expressed as a function of some coefficients $C$ and nondimensional velocity $v$:

In [ ]:
F_,v_, C_=sp.symbols("F' v' C")
F=sp.symbols("F")

eq = sp.Eq(F_, 
      C_*v_)
eq

This may look as a model where the force depend on the transverse velocity $v$ only, but this is actually not true. If this equation is converted converted back to SI units it is instead written:

In [ ]:
eq_F_prime = sp.Eq(F,prime_system.df_prime.loc['denominator','force']*F_)
eq_v_prime = sp.Eq(v,prime_system.df_prime.loc['denominator','linear_velocity']*v_)

eqs = [eq, eq_F_prime, eq_v_prime]

sp.Eq(F,sp.solve(eqs,F,F_,v_,dict=True)[0][F])

So it is now clear that the example force model above, also depends on the total velocity $U$. 

## Brix parameters
The hydrodynamic derivatives can be estimated with semi empirical formulas according to <cite id="ycziw">(Brix, 1993)</cite>.

In [ ]:
mask = df_parameters['prime'].notnull()
index = df_parameters.loc[mask,'prime'].index
coefficients=vmm.simulator.get_all_coefficients(sympy_symbols=False)
missing_coefficients = set(coefficients) - set(index)
missing_coefficients

In [ ]:
mask = df_parameters['prime'].notnull()
for index, parameter in df_parameters.loc[mask].iterrows():
    display(sp.Eq(parameter['symbol'],parameter['prime']))

In [ ]:
U_ = 2
glue("U_",U_);

In [ ]:
parameters=df_parameters['prime'].copy()

N_=30
df_captive_template = pd.DataFrame(index=np.arange(N_))
df_captive_template['U'] = U_
df_captive_template['u'] = df_captive_template['U']
df_captive_template['v'] = 0
df_captive_template['r'] = 0
df_captive_template['delta'] = 0

df_r = df_captive_template.copy()
df_r['r'] = np.linspace(-0.3,0,N_)
df_r['vary'] = 'r'

df_v = df_captive_template.copy()
df_v['v'] = np.linspace(0,0.5,N_)
df_v['u'] = np.sqrt(df_v['U']**2 - df_v['v']**2)
df_v['vary'] = 'v'

df_delta = df_captive_template.copy()
df_delta['delta'] = np.linspace(0.,0.4,N_)
df_delta['vary'] = 'delta'

df_captive = pd.concat([df_delta,df_v,df_r])

df_captive['X'] = run(vmm.simulator.X_qs_lambda, **parameters, **df_captive)
df_captive['Y'] = run(vmm.simulator.Y_qs_lambda, **parameters, **df_captive)
df_captive['N'] = run(vmm.simulator.N_qs_lambda, **parameters, **df_captive)
df_captive = ps.prime(df_captive, U=df_captive['U'])  # Convert to prime system

ncols = len(df_captive['vary'].unique())
fig,axes=plt.subplots(ncols=ncols, nrows=3)

vary_labels={
    'delta' : r'\delta',
}
for col,(vary, df_) in enumerate(df_captive.groupby(by='vary')):
    
    for row,dof in enumerate(['X','Y','N']):
        ax = axes[row,col]
        df_.plot(x=vary, y=dof, ax=ax)
        ax.get_legend().set_visible(False)
        ax.set_xlabel('')
        ax.grid(True)
        axes[row,0].set_ylabel(f"${dof}'$")
        
        ylims = [df_captive[dof].min(),df_captive[dof].max()]
        if ylims[0]!=ylims[1]:
            ax.set_ylim(ylims)
        
    axes[-1,col].set_xlabel(f"${vary_labels.get(vary,vary)}'$")

The figure above shows the quasi static forces for the present ship with the hydrodynamic derivatives during a variation of rudder angle $\delta$,
         yaw rate $r$ and tranverse velocity $v$ and total velocity $U$={glue:}`U_` m/s

## Simulation

### Decoupling
There is a coupling between the sway and yaw equation thruogh the added masses. These equations need to be decoupled to be usable in a simulation model. This is done by first expressing the equation in matrix form:

In [ ]:
X_eq = vmm.X_eq
Y_eq = vmm.Y_eq
N_eq = vmm.N_eq

A, b = sp.linear_eq_to_matrix([X_eq, Y_eq, N_eq], [u1d, v1d, r1d])

acceleration = sp.matrices.MutableDenseMatrix([u1d,v1d,r1d])
eq_simulator = sp.Eq(sp.UnevaluatedExpr(A)*sp.UnevaluatedExpr(acceleration),sp.UnevaluatedExpr(b))
eq_simulator

The decoupled equations are obtained by inverting the intertia matrix:

This equation can be written in a cleaner way if the $S$ term is introduced:

In [ ]:
A_inv = A.inv()
S = sp.symbols('S')
eq_S=sp.Eq(S,-sp.fraction(A_inv[1,1])[1])

A_inv_S = A_inv.subs(eq_S.rhs,S)
eq_acceleration_matrix_clean = sp.Eq(sp.UnevaluatedExpr(acceleration),sp.UnevaluatedExpr(A_inv_S)*sp.UnevaluatedExpr(b))
Math(vlatex(eq_acceleration_matrix_clean))

Where $S$ is a helper variable defined as:

In [ ]:
eq_S

### prime and SI
This concise expression to calculate the accelerations is expressed in the prime system. Conducting a simulation by integrating this expression will give a simulation that is conducted in the prime system, with prime system time etc., which is not very convenient. If a simulation should instead be conducted with a standard unit system (SI), the states (velocities as well as the resulting accelerations) and the ship parameters (length, masses and inertia) need to be converted. 

The input velocities in SI should be converted to prime and the output accelerations in prime needs to be converted back to SI:
1. Input: velocities : SI -> prime
2. Output: accelerations : prime -> SI

This can be done for each time step in the simulations, by implementing this logic into the code. Another alternative is to rewrite the expression into SI units, which will be done below. The damping functions: $X_D(u,v,r,\delta)$, $Y_D(u,v,r,\delta)$, $N_D(u,v,r,\delta)$ is also substituted with their polynomials to derive the full expression.

In [ ]:
u1d_function = sp.Function(r'\dot{u}')(u,v,r,delta)
v1d_function = sp.Function(r'\dot{v}')(u,v,r,delta)
r_function = sp.Function(r'\dot{r}')(u,v,r,delta)

subs_prime = [
    
    (m,m/prime_system.df_prime.mass.denominator),
    (I_z,I_z/prime_system.df_prime.inertia_moment.denominator),
    (x_G,x_G/prime_system.df_prime.length.denominator),
    
    (u, u/sp.sqrt(u**2+v**2)),
    (v, v/sp.sqrt(u**2+v**2)),
    (r, r/(sp.sqrt(u**2+v**2)/L)),
   
]

subs = [
(X_D, vmm.X_qs_eq.rhs),
(Y_D, vmm.Y_qs_eq.rhs),
(N_D, vmm.N_qs_eq.rhs),
]

subs = subs + subs_prime

A_SI = A.subs(subs)
b_SI = b.subs(subs)

x_dot = sympy.matrices.dense.matrix_multiply_elementwise(A_SI.inv()*b_SI,
                                                 sp.Matrix([(u**2+v**2)/L,(u**2+v**2)/L,(u**2+v**2)/(L**2)]))

sp.Eq(sp.UnevaluatedExpr(sp.Matrix([
    u1d_function,
    v1d_function,
    r_function,    
])),sp.UnevaluatedExpr(x_dot))


...so the expanded expression in SI units is a lot more complicated.

## Simulate data

In [ ]:
x, x1d = sp.symbols(r'\vec{x} \dot{\vec{x}}')  # State vector
h = sp.symbols('h')
u_input = sp.symbols(r'u_{input}')  # input vector
w_noise = sp.symbols(r'w_{noise}')  # input vector

f = sp.Function('f')(x,u_input,w_noise)
eq_system = sp.Eq(x1d, f)
eq_system

Where the state vector $\vec{x}$:

In [ ]:
eq_x = sp.Eq(x, sp.UnevaluatedExpr(sp.Matrix([x_0, y_0, psi, u, v, r])))
eq_x

In [ ]:
eq_x0_1d = sp.Eq(x_01d,u*sp.cos(psi)-v*sp.sin(psi))
eq_y0_1d = sp.Eq(y_01d,u*sp.sin(psi)+v*sp.cos(psi))
eq_psi_1d = sp.Eq(psi1d,r)

eq_f =sp.Eq(f,
sp.UnevaluatedExpr(
sp.Matrix([
    eq_x0_1d.rhs,
    eq_y0_1d.rhs,
    eq_psi_1d.rhs,
    u1d_function,
    v1d_function,
    r_function,
    
]
))
)
display(eq_f)

In [ ]:
x_ = sp.Matrix([u*sp.cos(psi)-v*sp.sin(psi),
                                u*sp.sin(psi)+v*sp.cos(psi),
                                r])

f_ = sp.Matrix.vstack(x_, x_dot)

subs = {value: key for key, value in p.items()}
subs[psi] = sp.symbols('psi')
lambda_f = lambdify(f_.subs(subs))

The actual simulation can be carried out with a simple Euler forward integration:

In [ ]:
def time_step(x_,u_):
    psi=x_[2]
    u=x_[3]
    v=x_[4]
    r=x_[5]
    delta = u_
    x_dot = run(lambda_f, **parameters, **ship_parameters, psi=psi, u=u, v=v, r=r, delta=delta).flatten()
    return x_dot

def simulate(x0,E, ws, t, us):
    
    simdata = np.zeros((6,len(t)))
    x_=x0
            
    for i,(u_,w_) in enumerate(zip(us,ws)):
       
        x_dot = time_step(x_,u_)
        
        x_=x_ + h_*x_dot
        
        simdata[:,i] = x_
                
    df = pd.DataFrame(simdata.T, columns=["x0","y0","psi","u","v","r"], index=t)
    df.index.name = 'time'
    df['delta'] = us
    
    return df

In [ ]:
parameters=df_parameters['prime'].copy()
N_ = 1000
t_ = np.linspace(0,50,N_)
h_ = float(t_[1]-t_[0])
ws = np.zeros(N_)
us = delta_ = np.deg2rad(20)*np.ones(N_)
x0_ = np.array([0,0,0,2,0,0])

E=np.array([0,0,0,1,1,1])
df = simulate(x0=x0_, E=E, ws=ws, t=t_, us=us)
df['beta'] = -np.arctan2(df['v'],df['u'])
df['U'] = np.sqrt(df['u']**2 + df['v']**2)
df.to_csv('first_simulation.csv', index=True)

In [ ]:
track_plot(
            df=df,
            lpp=ship_parameters["L"],
            beam=ship_parameters["B"],
            color="green",
        );

plot(df_result=df);

### Simulate parameter contributions

In [ ]:
df_result_prime = ps.prime(df, U=df['U'])

In [ ]:
X_ = sp.symbols('X_')
diff_eq_X = DiffEqToMatrix(ode=vmm.X_qs_eq.subs(X_D,X_), 
                                      label=X_, base_features=[delta,u,v,r])
X = diff_eq_X.calculate_features(data=df_result_prime)
X_parameters = df_parameters.groupby(by='dof').get_group('X')['prime'].dropna()
X_forces = X*X_parameters
X_forces.index = df_result_prime.index

Y_ = sp.symbols('Y_')
diff_eq_Y = DiffEqToMatrix(ode=vmm.Y_qs_eq.subs(Y_D,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])
X = diff_eq_Y.calculate_features(data=df_result_prime)
Y_parameters = df_parameters.groupby(by='dof').get_group('Y')['prime'].dropna()
Y_forces = X*Y_parameters
Y_forces.index = df_result_prime.index

N_ = sp.symbols('N_')
diff_eq_N = DiffEqToMatrix(ode=vmm.N_qs_eq.subs(N_D,N_), 
                                      label=N_, base_features=[delta,u,v,r])
X = diff_eq_N.calculate_features(data=df_result_prime)
N_parameters = df_parameters.groupby(by='dof').get_group('N')['prime'].dropna()
N_forces = X*N_parameters
N_forces.index = df_result_prime.index

Here is an interavtive graph showing how the various hydrodynamic derivatives contribute to the forces:

In [ ]:
display(px.line(X_forces, y=X_forces.columns, width=800, height=350, title='X'))
display(px.line(Y_forces, y=Y_forces.columns, width=800, height=350, title='Y'))
display(px.line(N_forces, y=N_forces.columns, width=800, height=350, title='N'))